In [15]:
import pandas as pd
import sys
import random
import re

from datasets import Dataset

sys.path.append("..")

### STS tryout

In [107]:
import torch

model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models', model='silero_te')
sentence_srb = "Posle toga su bugarski i vizantijski car sklopili savez protiv Srbije Srpske i bugarske snage su se sukobile 28 jula nedaleko od Velbužda Stefan Dečanski je imao najamnike iz Španije, koji neki nisu ni stigli na vreme, a bugarski car Mihajlo Šišman pomoćne odrede Tatara i Vlaha, a jedan odred je predvodio Dušan, za koga Danilov kaže da se proslavio u tom ratu Kasnije Dušan u svom zakoniku hvalisavo pominje ovu bitku Srpskom pobedom su otklonjene i opasnosti, u Bugarskoj je postavljen sin iz prvog braka poginulo cara i sestre Stefana Uroša III, a vizantijski car se povukao u Makedoniju, kada je saznao da su Srbi pobedili"
text = "Machine Learning is among the most popular and rapidly evolving technologies It is a field with a wide range of employment prospects"

apply_te(sentence_srb, lan='en')

Using cache found in /home/teodora/.cache/torch/hub/snakers4_silero-models_master


'Posle toga su Bugarski i Vizantijski car sklopili savez protiv Srbije, Srpske i bugarske snage su se sukobile 28 jula nedaleko od Velbu&da. Stefan De&anski je imao najamnike iz &panije, koji neki nisu ni stigli na vreme, a Bugarski car Mihajlo &i&man pomo&ne odrede Tatara i Vlaha, a jedan odred je predvodio Du&an, za koga Danilov ka&e da se proslavio u tom ratu Kasnije Du&an u svom zakoniku hvalisavo pominje ovu bitku Srpskom pobedom su otklonjene i opasnosti, u Bugarskoj je postavljen sin iz prvog braka Poginulo cara i sestre Stefana Uro&a [UNK] a Vizantijski car se povukao u Makedoniju, kada je saznao da su Srbi pobedili.'

In [5]:
# import spacy

# # Load the small Serbian model
# nlp = spacy.load('hr_core_news_sm')

# def split_sentences_spacy(text):
#     # Process the text using spaCy
#     doc = nlp(text)
#     # Extract sentences based on linguistic features
#     return [sent.text for sent in doc.sents]

# # Example Serbian text
# serbian_text_no_punctuation = """
# ja sam ucenik Ovo je testni tekst kako bih video Da li ce ovo raditi Ali nije uvek lako mislim da moze uspeti Ako se potrudim
# """

# sentences = split_sentences_spacy(sentence_srb)
# for sentence in sentences:
#     print(sentence)
#     print('---')
#     print(len(sentence))

# doc = nlp(sentence_srb)
# for token in doc:
#     print(token.text, token.pos_, token.dep_)


### Negaitves SentenceTransformers

In [7]:
pd.set_option('display.max_colwidth', None)  # None means no limit on column width
pd.set_option('display.width', 300)  

In [6]:
pd.read_parquet("../datasets/wiki.parquet")

,context,short_query,medium_query,long_query,scores
0,UTF-8 UTF-8 varijanta je najzgodnija za kodira...,UTF-8 kodiranje,Unicode fontovi za prenosivost teksta,Kako koristiti UTF-8 u Microsoft Word-u?,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
1,Osnovna kodna strana na personalnim računarima...,Kodna strana PC437,Podrška za jezike kodne strane,Kako Unicode rešava problem kodnih strana?,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
2,Njen naziv je UCS-2 zato što koristi dva oktet...,UCS-2 karakteri,UTF-16 i problem prostora,Kako UTF-8 rešava problem prenosa podataka?,"{'long_query': 5, 'medium_query': 5, 'short_qu..."
3,Ova transformaciona šema je prevashodno zgodna...,transformaciona šema latinica,MIME standardi za prenos fajlova,Kako funkcioniše 7-bitna transformaciona šema ...,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
4,Osnovna razlika je u tome što UTF-7 koristi sa...,UTF-7 Base64 prednosti,UCS-4 Unicode standard karakteristike,Zašto je UCS-4 plan za budućnost Unicode-a?,"{'long_query': 4, 'medium_query': 4, 'short_qu..."
...,...,...,...,...,...
659,"Ova igra, u kojoj nema surovosti, veoma je zg...",pravila rekreativne košarke,pravila takmičarske košarke u NBA,koji su uslovi za takmičarsku košarku,"{'long_query': 5, 'medium_query': 5, 'short_qu..."
660,Većina terena ima parket izrađen od drveta Že...,visina obruča koša,dimenzije terena za košarku,razlika u liniji za 3 poena NBA i međunarodna,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
661,Igrači su uniformisani i svi iz istog tima nos...,dresovi košarkaša,brojevi na dresovima košarkaša,koliko igrača sme biti na terenu,"{'long_query': 5, 'medium_query': 5, 'short_qu..."
662,"Zamene se mogu obavljati neograničeno, ali sam...",zamene sat zaustavljen,koliko traju četvrtine u NBA,koliko traju poluvremena na fakultetima,"{'long_query': 4, 'medium_query': 5, 'short_qu..."


In [47]:
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers import SentenceTransformer

ds = dataset = pd.read_parquet("../datasets/datasets/wiki_fixed.parquet")

model = SentenceTransformer("intfloat/multilingual-e5-small")

def convert_to_hf_dataset(dataframe: pd.DataFrame, question_type:str) -> Dataset:
    # Convert each InputExample into a dictionary
    data_dict = {
        "anchor": [],
        "positive": [],
    }
    for _, row in dataframe.iterrows():
        data_dict['anchor'].append(row[question_type])
        data_dict['positive'].append(row['context'])
    # Create a Hugging Face Dataset
    return Dataset.from_dict(data_dict)

In [50]:
dataset = convert_to_hf_dataset(ds, 'short_query')
dataset

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 7444
})

In [56]:
dataset_neg = mine_hard_negatives(
    dataset=dataset,
    model=model,
    range_min=10,
    range_max=100,
    max_score=0.8,
    margin=0,
    num_negatives=2,
    sampling_strategy="random",
    batch_size=128,
    use_faiss=True,
)

Found 7317 unique queries out of 7444 total queries.
Found an average of 1.017 positives per query.


Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/58 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


Metric       Positive       Negative     Difference
Count           7,444            837               
Mean           0.8608         0.7966         0.0623
Median         0.8617         0.7976         0.0644
Std            0.0233         0.0029         0.0253
Min            0.7630         0.7819        -0.0088
25%            0.8458         0.7958         0.0441
50%            0.8617         0.7976         0.0644
75%            0.8770         0.7985         0.0818
Max            0.9371         0.7996         0.1250
Skipped 37469 potential negatives (5.07%) due to the margin of 0.
Skipped 676371 potential negatives (96.41%) due to the maximum score of 0.8.
Could not find enough negatives for 14051 samples (94.38%). Consider adjusting the range_max, range_min, margin and max_score parameters if you'd like to find more valid negatives.


In [57]:
dataset_neg

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 837
})

In [41]:
df = pd.DataFrame(dataset_neg)

In [42]:
df

,anchor,positive,negative
0,razlozi odbijanja Kofi Anan plana,"Turska strana zastupa tezu o konfederalnoj zajednici država, dok grčka strana teži unitarizmu. Najozbiljniji pokušaj za ujedinjenje bio je 2004, a inicirao ga je generalni sekretar UN-a Kofi Anan. Ako bi se prihvatio taj plan celo ostrvo bi ušlo u EU, ako ne, samo južni, odnosno grčki. Grčki političari su procenili da ovaj sporazum više odgovara turskoj strani, pa su ga odbacili iz sledećih razloga: plan nije omogućavao povratak svih grčkih izbeglica, kao ni vraćanje svog zemljišta koje im je pre pripadalo, Turci bi dobili previše mesta u organima vlasti i plan takođe nije uključivao uklanjanje britanskih baza sa ostrva. Iz tih razloga, na referendumu na grčkom delu ostrva 75 % građana je odbilo sporazum. Iako je plan prihvatilo 64 % građana Severnog Kipra, on nije prošao, tako da je nastao pravi paradoks - u EU je praktično ušao samo južni deo, iako službeno Turska Republika Severnog Kipra predstavlja deo Kiparske Republike pa samim tim je deo EU. Velika prepreka ujedinjenju ostrva je stalna prisutnost 50.000 turskih vojnika. Službena površina Kiparske Republike koja stoji u svim svetskim podacima je u stvari površina ostrva koja iznosi 9.251 -{km²}-.","U celoj Indiji ulice su bile blokirane Indijcima koji su sedili na njima, a kada su se odbili ustati policija ih je tukla Gandi je bio uhapšen, ali je uskoro zbog pritiska javnosti bio pušten U okviru borbe za ekonomsku nezavisnost Indije, uključujući bojkot britanske robe, Gandi je osmislio svaraj (sanskritski: samovladavina) pokret Ekonomski aspekti pokreta bili su veoma važni jer je iskorištavanje indijskih seljaka od strane britanskih industrijalaca dovodilo do siromašenja sela i destrukcije kućnog zanatstva"
1,razlozi odbijanja Kofi Anan plana,"Turska strana zastupa tezu o konfederalnoj zajednici država, dok grčka strana teži unitarizmu. Najozbiljniji pokušaj za ujedinjenje bio je 2004, a inicirao ga je generalni sekretar UN-a Kofi Anan. Ako bi se prihvatio taj plan celo ostrvo bi ušlo u EU, ako ne, samo južni, odnosno grčki. Grčki političari su procenili da ovaj sporazum više odgovara turskoj strani, pa su ga odbacili iz sledećih razloga: plan nije omogućavao povratak svih grčkih izbeglica, kao ni vraćanje svog zemljišta koje im je pre pripadalo, Turci bi dobili previše mesta u organima vlasti i plan takođe nije uključivao uklanjanje britanskih baza sa ostrva. Iz tih razloga, na referendumu na grčkom delu ostrva 75 % građana je odbilo sporazum. Iako je plan prihvatilo 64 % građana Severnog Kipra, on nije prošao, tako da je nastao pravi paradoks - u EU je praktično ušao samo južni deo, iako službeno Turska Republika Severnog Kipra predstavlja deo Kiparske Republike pa samim tim je deo EU. Velika prepreka ujedinjenju ostrva je stalna prisutnost 50.000 turskih vojnika. Službena površina Kiparske Republike koja stoji u svim svetskim podacima je u stvari površina ostrva koja iznosi 9.251 -{km²}-.","ona ponovo obratila ministarstvu unutrašnjih dela, sa veoma jasnim stavom i potpuno rezolutnim tonom koji nije bio molba već zahtev ministru """"da ima sva lekarska prava data svim lekarima, bez razlike pola, sanitetskim zakonomiz prakse dosadašnje gospode Ministara izlazi da ja mogu kao lekar služiti državu i preko trideset godina državne službe i da ni u položaju, ni u plati ne krenem ni malo dalje"""" Pošto je njen stav i zahtev sada bio odlučniji pozvan je načelnik Saniteta da i on iskaže svoje mišljenje"
2,Beogradska biblioteka pre Prvog svetskog rata,"000 članova Još od vremena kada je Beograd postao prestonica javljali su se zahtevi da Opština grada Beograda podigne Narodni dom u kome bi bile smeštene čitaonica i biblioteka Pre Prvog svetskog rata postojala je u Opštini stručna biblioteka, koja je od 1929 godine bila smeštena u jednoj sobi i služila za službene potrebe ove ustanove Od 1922 do 1929 godine u budžetu je izdvajan novac koji je trebalo da omogući da se ova biblioteka sredi i preuzme funkciju javne bib

### Punkt 

In [9]:
wiki = pd.read_parquet("../datasets/wiki.parquet")
wiki.head(3)

,context,short_query,medium_query,long_query,scores
0,"UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet. Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita). To je bio tzv. ""-{ASCII}-"" ili ""-{US-ASCII}-"" standard. 1968. godine je skup karaktera proširen na 256 (8 bita), a ""gornjih"" 128 karaktera je bilo korišćeno za dodatne karaktere. Iz neke navike je i ovaj prošireni ""-{ASCII}-"" nazivan ""-{ASCII}-"", tako da tu često dolazi do zabune. Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane (""-{Code Page}-"") koje definišu ponašanje tog dodatnog skupa slova.",UTF-8 kodiranje,Unicode fontovi za prenosivost teksta,Kako koristiti UTF-8 u Microsoft Word-u?,"{'long_query': 5, 'medium_query': 4, 'short_query': 5}"
1,"Osnovna kodna strana na personalnim računarima (""-{PC437}-"") u tom gornjem setu karaktera definiše razne grafičke karaktere za crtanje tekstualnih prozora i slično. Kasnije je razvijeno još puno kodnih strana koje podržavaju određene jezike. Tako postoje ""-{Latin1}-"" (""-{ISO-8859-1}-"") za latinična pisma Zapadne Evrope (Francuska, Nemačka, Španija, ...), ""-{Latin2}-"" (""-{ISO-8859-2}-"") i ""-{Windows-1250}-"" za latinična pisma Istočne Evrope (srpska latinica i sl.), ""-{ISO-8859-5}-"", ""-{KOI8-R}-"" i ""-{Windows-1251}-"" za ćirilicu, itd. Osnovni problem sa kodnim stranama je to što se međusobno isključuju, tj. ceo dokument mora da bude napisan istim pismom. To uglavnom nije problem realizovati, ali ako bi bilo potrebno pomešati dva pisma, kao na primer u nekom turističkom vodiču gde zajedno postoji i tekst na srpskom, na engleskom i na francuskom, nailazi se na problem. Zbog toga se došlo do ideje da se napravi jedinstveni zapis za sve jezike - Junikod (""-{Unicode}-"") (""-{united}-"" - ujedinjen, zajednički; ""-{code}-"" - kod, zapis). Postoji više verzija Unicode-a. Bazična verzija je dvobajtni format zapisa do 2 = 65536 karaktera.",Kodna strana PC437,Podrška za jezike kodne strane,Kako Unicode rešava problem kodnih strana?,"{'long_query': 5, 'medium_query': 4, 'short_query': 5}"
2,"Njen naziv je UCS-2 zato što koristi dva okteta, odnosno dva bajta. Sa tih 65536 karaktera rešen je problem zapisa skoro svih postojećih pisama (uključujući čak i neka izmišljena, kao na primer klingonsko pismo). Ovaj tip Unicode-a se naziva Plain UCS-2 ili UTF-16. Sada se javlja problem alokacije prostora za Unicode poruku na medijumu koji se koristi. Ako je reč o nekom dokumentu na disku, on će da zauzima duplo više prostora nego konvencionalan dokument jer će se svaki karakter zapisivati sa dva bajta umesto samo sa jednim. Ako je reč o prenosu podataka preko računarske mreže, biće potrebno preneti duplo više podataka, pa će samim tim i prenos da traje duplo više (odnosno da košta duplo više). Postavlja se pitanje da li je to suviše velika cena za univerzalno pismo i da li postoji neki način da se taj problem prevaziđe i izbegne. Kao rešenje uvek stoji mogućnost da se zapisuje nekom odgovarajućom kodnom stranicom i troši bajt po karakteru, ako nije neophodno korišćenje više pisama u istom dokumentu (što se retko dešava). Drugo rešenje je korišćenje tzv. transformacionih šeme za pogodniji zapis i prenos podataka korišćenjem Unicode-a. Prvo je razvijena Unicode transformaciona šema sa osnovnom jedinicom od 8 bita (UTF-8). Pomoću nje se karakter zapisuje u jednom, dva ili tri bajta, u zavisnosti od toga o kom je karakteru reč.",UCS-2 karakteri,UTF-16 i problem prostora,Kako UTF-8 rešava problem prenosa podataka?,"{'long_query': 5, 

In [10]:
contexts = wiki['context'].tolist()
contexts[:3]

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet. Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita). To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard. 1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere. Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune. Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje tog dodatnog skupa sl

In [11]:
data = pd.read_parquet("../datasets/TRAIN11k_fixed_v2.parquet")
data.shape

(11714, 7)

In [12]:
contexts_all = data['context'].tolist()
print(contexts_all[:2])
print(len(contexts_all))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita)', ' "-{ASCII}-" ili "-{US-ASCII}-" standard 1968 godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune Da bi postojala podrška za više jezika, smišljane su tzv kodne strane ("-{Code Page}-") koje definišu ponašanje tog dodatnog skupa slova Osnovna kodna st

In [201]:
positives = []
string = "."
for i in contexts_all:
    if len(i.split(".")) > 1:
        positives.append(i.split(".")[0])
    else:
        words = i.split(" ")
        positive = " ".join(words[: len(words) // 4])
        positives.append(positive)

In [56]:
def get_all_capitalized_and_following(texts, sample_size):
    results = []
    samples = []
    for text in texts:
        rand = random.randint(7, 18)
        pattern = r'([A-Z][a-z]*)\s(\S+\s){0, rand}'
        matches = re.finditer(pattern, text)
        result = [match.group(0) for match in matches]
        
        if not result:
            words = text.split()[:rand]  
            result = [' '.join(words)]
            
        results.append(result)
    
    for res in results:
        sampled = random.sample(res, sample_size)
        samples.extend(sampled)
    return samples

In [28]:
hundred = contexts_all[:100]
print(len(hundred))

100


In [57]:
result = get_all_capitalized_and_following(contexts_all, 1)
print(result[:10])

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta Dato je', '"-{ASCII}-" ili "-{US-ASCII}-" standard 1968 godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128', 'Tako postoje "-{Latin1}-" ("-{ISO-8859-1}-") za latinična pisma Zapadne Evrope (Francuska, Nemačka,', 'Njen naziv je UCS-2 zato što koristi dva okteta, odnosno', 'Ova transformaciona šema je prevashodno zgodna za', 'Tu se karakter zapisuje u jednom ili u nekoliko bajtova, slično kao i', 'Taj noviji format je više napravljen kao plan za budućnost nego kao realna', 'Procesorski element izvodi aritmetičke i logičke operacije, a', 'Originalno oni su bili veličine velike sobe, konzumirajći količinu energije jednaku količini', 'Međutim, ugrađeni računari prisutni u mnogim uređajima od -{MP3}- plejera do borbenih aviona i od igračaka do']


In [40]:
new11k = pd.DataFrame({'contexts' : contexts_all,
                       'query': [i for i in result]})

In [35]:
new11k[10000:]

,contexts,query
10000,"Tako će pred uvođenje letnjeg reda vožnje svih sto autobusa predviđenih ugovorom biti na ulicama Beograda. Solo autobus košta 180.000, a zglobni 220.000 evra. „Ikarbus“ je na autobuse dao dve i po godine garancije, kao i još dve i po godine u postgarancijskom servisu, što znači da GSP „Beograd“ neće morati da ih servisira narednih pet godina. Koncept „Beobus“ je u maju prošle godine prezentovan na sajmu u Veroni, gde je privukao veliku pažnju i dobio izuzetne kritike.",P „Beograd“ neće morati da ih servisira narednih pet godina. Koncept „Beobus“ je u maju prošle godine prezentovan na
10001,Skandal neviđenih razmera Dekan Filozofskog fakulteta u Beogradu Aleksandar Kostić izjavio je za Danas da će taj fakultet preduzeti sve neophodne korake da se ispita ko je odgovoran za to što se u utorak uveče obrušio deo plafona u jednoj od učionica tokom predavanja Kostić je ocenio da je reč o „skandalu neviđenih razmera“ i da je „posebno zabrinjavajuće to što su nečijom nemarnošću bili ugroženi životi i bezbednost studenata“ Dekan Filozofskog fakulteta u Beogradu Aleksandar Kostić izjavio je za Danas da će taj fakultet preduzeti sve neophodne korake da se ispita ko je odgovoran za to što se u utorak uveče obrušio deo plafona u jednoj od učionica tokom predavanja,Dekan Filozofskog fakulteta u Beogradu Aleksandar Kostić izjavio je za Danas da će taj fakultet preduzeti sve neophodne korake
10002,"– Formirali smo komisiju koja treba da utvrdi zašto se plafon obrušio i ko je imenom i prezimenom za to odgovoran, a nakon toga će fakultet najverovatnije preduzeti pravne korake Prekosutra bi trebalo da bude održana sednica Saveta fakulteta i očekujemo da će dotad izveštaj komisije biti završen O tome će biti reči i na nekoj od narednih sednica Senata Univerziteta, jer ovo što se dogodilo može da se ponovi i u nekoj školi ili na nekom drugom fakultetu – rekao je profesor Kostić",Formirali smo komisiju koja treba da utvrdi zašto se plafon obrušio i ko je imenom i prezimenom za to
10003,"Brojne manifestacije su nešto što daje poseban pečat letnjim dešavanjima na Adi Samo u toku prošle godine bilo ih je više od sedamdeset Ove godine Ada Ciganlija biće domaćin nekoliko svetskih sportskih skupova, kao što je Svetsko studentsko prvenstvo u veslanju i u kajaku i drugi sportski događaji Posebna atrakcija biće „Avantura park“ gde će posetioci moći da se uz pomoć instruktora prebacuju, poput Tarzana, sa drva na drvo, uz pomoć lijana i mostića Inače, najmlađa populacija je do sada bila nekako u senci, pa će od ovog leta biti otvorena dva nova igrališta za najmlađe posetioce Ade Ciganlije","Posebna atrakcija biće „Avantura park“ gde će posetioci moći da se uz pomoć instruktora prebacuju, poput Tarzana, sa drva"
10004,"U prisustvu porodice, predsednika Srbije Borisa Tadića, ministara u Vladi Srbije, Zorana Alimpića, predsednika Skupštine grada i vd gradonačelnika Beograda pomenu je prisustvovao i veliki broj prijatelja i građana Beograda U Aleji zaslužnih građana na Novom groblju u subotu je održan šestomesečni pomen Nenadu Bogdanoviću, gradonačelniku Beograda U prisustvu porodice, predsednika Srbije Borisa Tadića, ministara u Vladi Srbije, Zorana Alimpića, predsednika Skupštine grada i v","U prisustvu porodice, predsednika Srbije Borisa Tadića, ministara u Vladi Srbije, Zorana Alimpića, predsednika Skupštine grada i vd gradonačelnika"
...,...,...
11709,"Čekmedžijić je osobito navaljivao, pa mora i čika-Gavra. Svi se digoše. Najpre ode Pinterić i Čarukdžić, počem se sa čika-Gavrom i Ljubom oprostiše. Sad se opet Ljuba i čika-Gavra praštaju. Pitaju za frajlu, no ona ne može doći. Kaže sluškinja da joj je zlo, leži. Lepo se oprostiše i odoše. Posle po sata je Čekmedžijić sa čika-Gavrom na putu. Gospođa Makra zadovoljna je bila sa društvom; bar se nasmejala, uzela je čika-Gavru na mindros. Ode Paulini da vidi šta radi. Paulina leži na divanu.","Svi se digoše. Najpre ode Pinterić i Čarukdžić, počem se sa čika-Gavrom i Ljubom oprostiše. Sad se

In [52]:
new11k['is_match'] = new11k.apply(lambda row: row['contexts'].startswith(row['query']), axis=1)
new11k.head()

,contexts,query,is_match
0,"UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita)",Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet Prvi računari,False
1,"""-{ASCII}-"" ili ""-{US-ASCII}-"" standard 1968 godine je skup karaktera proširen na 256 (8 bita), a ""gornjih"" 128 karaktera je bilo korišćeno za dodatne karaktere Iz neke navike je i ovaj prošireni ""-{ASCII}-"" nazivan ""-{ASCII}-"", tako da tu često dolazi do zabune Da bi postojala podrška za više jezika, smišljane su tzv kodne strane (""-{Code Page}-"") koje definišu ponašanje tog dodatnog skupa slova Osnovna kodna strana na personalnim računarima (""-{PC437}-"") u tom gornjem setu karaktera definiše razne grafičke karaktere za crtanje tekstualnih prozora i slično","Code Page}-"") koje definišu ponašanje tog dodatnog skupa slova Osnovna kodna strana na personalnim računarima (""-{PC437}-"") u tom gornjem",False
2,"Tako postoje ""-{Latin1}-"" (""-{ISO-8859-1}-"") za latinična pisma Zapadne Evrope (Francuska, Nemačka, Španija, ), ""-{Latin2}-"" (""-{ISO-8859-2}-"") i ""-{Windows-1250}-"" za latinična pisma Istočne Evrope (srpska latinica i sl), ""-{ISO-8859-5}-"", ""-{KOI8-R}-"" i ""-{Windows-1251}-"" za ćirilicu, itd Osnovni problem sa kodnim stranama je to što se međusobno isključuju, tj ceo dokument mora da bude napisan istim pismom","Evrope (srpska latinica i sl), ""-{ISO-8859-5}-"", ""-{KOI8-R}-"" i ""-{Windows-1251}-"" za ćirilicu, itd Osnovni problem sa kodnim stranama je to",False
3,"Njen naziv je UCS-2 zato što koristi dva okteta, odnosno dva bajta. Sa tih 65536 karaktera rešen je problem zapisa skoro svih postojećih pisama (uključujući čak i neka izmišljena, kao na primer klingonsko pismo). Ovaj tip Unicode-a se naziva Plain UCS-2 ili UTF-16. Sada se javlja problem alokacije prostora za Unicode poruku na medijumu koji se koristi. Ako je reč o nekom dokumentu na disku, on će da zauzima duplo više prostora nego konvencionalan dokument jer će se svaki karakter zapisivati sa dva bajta umesto samo sa jednim. Ako je reč o prenosu podataka preko računarske mreže, biće potrebno preneti duplo više podataka, pa će samim tim i prenos da traje duplo više (odnosno da košta duplo više). Postavlja se pitanje da li je to suviše velika cena za univerzalno pismo i da li postoji neki način da se taj problem prevaziđe i izbegne. Kao rešenje uvek stoji mogućnost da se zapisuje nekom odgovarajućom kodnom stranicom i troši bajt po karakteru, ako nije neophodno korišćenje više pisama u istom dokumentu (što se retko dešava). Drugo rešenje je korišćenje tzv. transformacionih šeme za pogodniji zapis i prenos podataka korišćenjem Unicode-a. Prvo je razvijena Unicode transformaciona šema sa osnovnom jedinicom od 8 bita (UTF-8). Pomoću nje se karakter zapisuje u jednom, dva ili tri bajta, u zavisnosti od toga o kom je karakteru reč.","Ako je reč o prenosu podataka preko računarske mreže, biće potrebno preneti duplo više podataka, pa će samim tim",False
4,"Ova transformaciona šema je prevashodno zgodna za upotrebu u jezicima koji koriste latinicu O UTF-8 će biti više reči u poglavlju ""Ukratko o UTF-8"" Jedan deo Mail Transfer Agent-a, kao i zvanični standard za Internet poštu (IETF: STD 11, RFC 822) podržava samo 7-bitne mail poruke MIME standardi (RFC 2045, RFC 2046, RFC 2047, RFC 2048 i RFC 2049) omogućavaju prenos višebitnih reči preko Internet mail-a, koristeći Base64 i Quoted Printable načine kodiranj

In [55]:
new11k['is_match'].value_counts()

is_match
False    10242
True      1472
Name: count, dtype: int64

In [59]:
import numpy as np

lens = []
for i in new11k['query']:
    length = len(i.split(" "))
    lens.append(length)

print(lens[:100])
print(np.mean(lens))

[20, 20, 20, 20, 20, 12, 20, 20, 19, 20, 20, 10, 20, 20, 20, 20, 20, 19, 8, 20, 20, 20, 20, 12, 20, 20, 20, 20, 20, 20, 20, 20, 20, 16, 20, 20, 20, 18, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 8, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 3, 20, 20, 5, 20, 20, 11, 20, 20, 20, 20, 20, 20, 20, 20, 16, 20, 20, 20, 20, 20, 20, 20, 20, 16, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
18.831227590916853


In [63]:
new11k.to_parquet('../datasets/TRAIN11k_test_query.parquet')

In [19]:
len(contexts_all) == len(result)

True

In [287]:
# Get indices of empty sublists
empty_sublists_indices = [i for i, sublist in enumerate(result) if not sublist]

# Print indices
print(f"Indices of empty sublists: {empty_sublists_indices}")


Indices of empty sublists: []


In [284]:
contexts_all[305]

'1897 1897 je bila prosta godina. 1896 1896. je bila prestupna godina. 1893 1893 je bila prosta godina. 1892 1892 je bila prestupna godina. 1891 1891. je bila prosta godina. 1890 1890. je bila prosta godina. 1951 1951 je bila prosta godina. 1952 1952. je bila prestupna godina. 1953 1953. je bila prosta godina. 1954 1954. je bila prosta godina. 1955 1955. je bila prosta godina. 1956 1956. je bila prestupna godina. 1957 1957. je bila prosta godina. 1940 1940. je bila prestupna godina. 1942 1942. je bila prosta godina. 1946 1946. je bila prosta godina. 1948 1948. je bila prestupna godina. 1949 1949. je bila prosta godina. 1930 1930. je bila prosta godina.'

In [239]:
positives = []
res = []
string = "."
sample_size = 1
for i in contexts_all:
    if len(i.split(".")) > 1:
        positives.append(i.split(".")[0])
    else:
        without_punct = get_all_capitalized_and_following(i)
        res.append(random.sample(without_punct, 1))


# others = get_all_capitalized_and_following(contexts, sample_size=sample_size)
# positives.extend(others)

ValueError: Sample larger than population or is negative

In [204]:
from pprint import pprint


pprint(positives[:30])
print(len(positives))

['Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, '
 'Netscape Composer-u i tekstualnom editoru Kate U ',
 'Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na '
 'računar ili za objavljivanje teksta na Internet Prvi računari ',
 'Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da '
 'tu često dolazi do zabune Da bi ',
 'Code Page}-") koje definišu ponašanje tog dodatnog skupa slova Osnovna kodna '
 'strana na personalnim računarima ("-{PC437}-") u tom gornjem ',
 'Tako postoje "-{Latin1}-" ("-{ISO-8859-1}-") za latinična pisma Zapadne '
 'Evrope (Francuska, Nemačka, Španija, ), "-{Latin2}-" ("-{ISO-8859-2}-") i '
 '"-{Windows-1250}-" za latinična ',
 'Evrope (srpska latinica i sl), "-{ISO-8859-5}-", "-{KOI8-R}-" i '
 '"-{Windows-1251}-" za ćirilicu, itd Osnovni problem sa kodnim stranama je '
 'to ',
 'Njen naziv je UCS-2 zato što koristi dva okteta, odnosno dva bajta',
 'Ova transformaciona šema je prevashodno zgod

In [206]:
sentence_srb = "Posle toga su bugarski i vizantijski car sklopili savez protiv Srbije Srpske i bugarske snage su se sukobile 28 jula nedaleko od Velbužda Stefan Dečanski je imao najamnike iz Španije, koji neki nisu ni stigli na vreme, a bugarski car Mihajlo Šišman pomoćne odrede Tatara i Vlaha, a jedan odred je predvodio Dušan, za koga Danilov kaže da se proslavio u tom ratu Kasnije Dušan u svom zakoniku hvalisavo pominje ovu bitku Srpskom pobedom su otklonjene i opasnosti, u Bugarskoj je postavljen sin iz prvog braka poginulo cara i sestre Stefana Uroša III, a vizantijski car se povukao u Makedoniju, kada je saznao da su Srbi pobedili"

get_all_capitalized_and_following(sentence_srb)

['Kasnije Dušan u svom zakoniku hvalisavo pominje ovu bitku Srpskom pobedom su otklonjene i opasnosti, u Bugarskoj je postavljen ']

In [181]:
from datasets import load_dataset

ds = load_dataset("jerteh/SrpWiki")
data = ds['train']['text']

In [182]:
len(data)

3818535

In [183]:
subset = data[:1000]
len(subset)

1000

In [189]:
import numpy as np
lengths = []
for i in data:
    length = len(i.split(" "))
    lengths.append(length)

print(lengths[:5])
print(len(lengths))

[10, 18, 23, 37, 8]
3818535


In [190]:
np.mean(lengths)

18.17924439608384

In [75]:
string = "Igre koje su se temeljile na šutiranju lopte odigravale su se u mnogim zemljama tokom istorije."
string_a = "Za to vreme kroz grad prolaze brojni krstaški pohodi na Istok ostavljajući svoj rušilački pečet na njemu."
for idx, sentence in enumerate(data):
    if sentence in string_a:
        print(idx)
        print(sentence)
    elif sentence in string:
        print(sentence)
        print(idx)

Igre koje su se temeljile na šutiranju lopte odigravale su se u mnogim zemljama tokom istorije.
4518
30331
Za to vreme kroz grad prolaze brojni krstaški pohodi na Istok ostavljajući svoj rušilački pečet na njemu.


In [87]:
def return_dic(sentences : List[str]) -> List[Dict[str, str|int]]:
    """
    Generates a list of dictionaries from a list of sentences.

    Each dictionary contains a sentence and its corresponding index in the original list.

    Parameters:
    sentences (List[str]): A list of sentences.

    Returns:
    List[Dict[str, str|int]]: A list where each element is a dictionary with keys:
        - 'sentence' [str]: The original sentence.
        - 'id' [int]: The index of the sentence in the input list.
         Example:
        >>> texts = ["Sentence 1" "Sentence 2"]
        >>> sentences_dic = return_dic(texts)
        >>> print(sentences_dic)
    """
    sent_lst_dic = [{'sentence': x, 'id' : i} for i, x in enumerate(sentences)]
    return sent_lst_dic

In [78]:
from typing import Dict, List


def combine_sentences(sentences: List[Dict[str,str]], buffer_size: int) -> List[Dict[str,str]]:
    """
    Combines each sentence in a list with its surrounding sentences based on a buffer size.

    For each sentence, the function concatenates the sentences within the specified buffer size before and after it, 
    creating a new key 'combined_sentence' in each dictionary.

    Parameters:
    sentences (List[Dict[str, str]]): A list of dictionaries, each containing a 'sentence' key.
    buffer_size (int): The number of sentences before and after the current sentence to include in the combination.

    Returns:
    List[Dict[str, str]]: The updated list of dictionaries with each dictionary containing an additional 'combined_sentence' key.
     Example:
    >>> texts = ["Sentence 1", "Sentence 2"]
    >>> sentences_dic = return_dic(texts)
    >>> print(sentences_dic)
    """
    for i in range(len(sentences)):
        combined_sentence = ''

        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '
        combined_sentence += sentences[i]['sentence']

        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']

        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

In [88]:
dictionary = return_dic(subset)

In [89]:
dictionary

[{'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.',
  'id': 0},
 {'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.',
  'id': 1},
 {'sentence': 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.',
  'id': 2},
 {'sentence': 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).',
  'id': 3},
 {'sentence': 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.',
  'id': 4},
 {'sentence': '1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere.',
  'id': 5},
 {'sentence': 'Iz neke navike je i ovaj prošireni "-{ASCII}-" naz

In [92]:
combined = combine_sentences(dictionary, 2)
combined

[{'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.',
  'id': 0,
  'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.'},
 {'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.',
  'id': 1,
  'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za obja